In [ ]:
import time
from datetime import datetime, timedelta
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import json


# Funkcja do generowania URL dla danej daty
def generate_url_for_date(date):
    return f"https://tt.league-pro.com/tours/?year={date.year}&month={date.month:02d}&day={date.day:02d}"

def extract_links(soup):
    # Znajdowanie dat w elemencie span w div o klasie "node-desc item"
    return [
            "https://tt.league-pro.com/" + link["href"]
            for table in soup.find_all("table", class_="table")
            for link in table.find_all("a", string=lambda text: "Tournament" in text if text else False)
        ]

def get_tournament_links_from_calendar_day(url):
    """
    Pobiera wszystkie linki do turniejów z kalendarza dnia na podanym URL.

    Args:
        url (str): URL strony z kalendarzem dnia.

    Returns:
        list: Lista linków do turniejów, jeśli strona została pomyślnie pobrana i sparsowana.
        str: Komunikat błędu w przypadku problemów z żądaniem HTTP.
    """
    try:
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36"
        }
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            content = response.content
            soup = BeautifulSoup(content, "lxml")
            links = extract_links(soup)
            return links
        else:
            return f"Request failed with status code {response.status_code} for URL {url}."            
    except requests.exceptions.RequestException as e:
        return f"An error occurred while fetching URL {url}: {e}"


def get_html_from_page(link):
    """
     Funkcja wykonuje zapytanie HTTP do podanego URL, pobiera zawartość strony
    i zwraca ją w postaci obiektu BeautifulSoup.

    Args:
        url (str): URL strony turnieju, z którego chcemy pobrać HTML.

    Returns:
        BeautifulSoup: Obiekt BeautifulSoup zawierający załadowany HTML turnieju,
                       umożliwiający dalszą analizę.
        str: Komunikat o błędzie, jeśli zapytanie zakończy się niepowodzeniem.
    """
    try:
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36"
        }
        response = requests.get(link, headers=headers)
        if response.status_code == 200:
            content = response.content
            soup = BeautifulSoup(content, "lxml")
            return soup
        else:
            return f"Request failed with status code {response.status_code} for URL {url}."
    except requests.exceptions.RequestException as e:
        print(f"An error occurred while fetching URL {link}: {e}")


def scrape_match_data(soup):
    """
    Scrapuje dane meczow ze strony zakonczonego turnieju
    
    Args:
        BeautifulSoup: Obiekt BeautifulSoup zawierający załadowany HTML turnieju,
                       umożliwiający dalszą analizę.

    Returns:
        list's: Tablice poszczegolnych parametrow, z brakujacym szczegolowym przebiem meczu. Jezeli turniej jest 4-osobowy,
                to dlugosc tablic to 8, jezeli 5-osobowy to dlugosc wynosi 12.
    """
    elo_lewego = []
    delta_elo = []
    godzina = []
    imie_lewego = []
    imie_prawego = []
    wynik_setowy = []
    exact_scores = [] 
    elo_prawego = []
    url_spotkania = []
    roznica_elo = []
    winner = []
    
    content_div = soup.find_all("table", class_="games_list")
    table = content_div[0]
    rows = table.find_all("tr")
    
    name = extract_tournament_name(soup)
    tournament_date = extract_time_start_tournament(soup)[0] if extract_time_start_tournament(soup) else "BRAK"

    for row in rows:
        cells = row.find_all("td")
        if len(cells) > 2:
            player_left = cells[1].find("a", href=lambda x: x and "players" in x)
            if player_left:
                imie_lewego.append(player_left.text.strip())
            
            wynik = cells[1].find("a", class_="undrr bold")
            if wynik:
                wynik_setowy.append(wynik.text.strip()) 
    
            if wynik and wynik.name == 'a' and 'href' in wynik.attrs:#zle
                url_spotkania.append(f"https://tt.league-pro.com/{wynik['href']}")
            #print(cells)
    
            godzina_meczu = cells[0].find("a", class_="undr")
            if godzina_meczu:
                parsed_datetime = parse_datetime(tournament_date, godzina_meczu.text.strip())
                if parsed_datetime:
                    godzina.append(parsed_datetime)
           
            delta = cells[2].find("small")
            if delta:
                delta_float = convert_to_float(delta.text.strip())
                if delta_float > 0:
                    winner.append(1)
                    delta_elo.append(delta.text.strip()) 
                else:
                    winner.append(0)
                    delta_elo.append(delta.text.strip()) 
    
            exact_scores.extend(
                (x.text.replace("(", "").replace(")", "") for idx, cell in enumerate(cells) 
                if (x := cell.find("small", class_="nowrap")) and idx % 2 != 0)
            )
                        
            elo_prawego.extend(
                x.text for idx, cell in enumerate(cells) 
                if (x := cell.find("b", class_="small")) and idx % 2 != 0
            )
            elo_lewego.extend(
                x.text for idx, cell in enumerate(cells) 
                if (x := cell.find("b", class_="small")) and idx % 2 == 0
            )
    
            imie_prawego.extend(
                x.text for idx, cell in enumerate(cells) 
                if (x := cell.find("a", href=lambda href: href and "players" in href)) and idx % 2 == 0
            )
    return godzina, name[0], url_spotkania, imie_lewego, elo_lewego, delta_elo, wynik_setowy, exact_scores, elo_prawego, imie_prawego, winner, elo_diff 


def extract_time_start_tournament(soup):
    # Znajdowanie dat w elemencie span w div o klasie "node-desc item"
    return [
        span.text.strip()
        for div in soup.find_all("div", class_="node-desc item")
        for span in div.find_all("span")
    ]

def extract_tournament_name(soup):
    # Znajdowanie nazw turniejów w elemencie h1
    return [
        h1.text.strip()
        for div in soup.find_all("div", class_="node-page page")
        for node in div.find_all("div", class_="node")
        for h1 in node.find_all("h1")
    ]

def calculate_elo_diff(elo_lewego, elo_prawego):
    """
    Oblicza różnicę pomiędzy odpowiadającymi sobie elementami dwóch tablic.
    
    Args:
        elo_lewego (list): Tablica ELO lewego zawodnika.
        elo_prawego (list): Tablica ELO prawego zawodnika.

    Returns:
        list: Tablica różnic lewego - prawego ELO.
    """
    # Sprawdzamy długość tablic
    if len(elo_lewego) != len(elo_prawego):
        print("Ostrzeżenie: różne liczby elementów w 'elo_lewego' i 'elo_prawego'. Wyniki będą obcięte do wspólnej długości.")
    
    # Obliczamy różnice
    elo_diff = [
        int(elo_lewego[i]) - int(elo_prawego[i])
        for i in range(min(len(elo_lewego), len(elo_prawego)))
    ]
    
    return elo_diff

def convert_to_float(text):
    """
    Konwertuje ciąg znaków (str) na liczbę zmiennoprzecinkową (float) na podstawie określonych zasad.

    Funkcja sprawdza, czy ciąg znaków reprezentuje liczbę z możliwym znakiem '+' lub '-',
    albo równą '0'. W przypadku poprawnego formatu zwraca liczbę zmiennoprzecinkową.
    W przeciwnym razie zgłasza wyjątek ValueError.

    Args:
        text (str): Ciąg znaków do konwersji.

    Returns:
        float: Liczba zmiennoprzecinkowa odpowiadająca podanemu ciągowi znaków.

    Raises:
        ValueError: Jeśli ciąg nie zawiera poprawnego formatu liczby.
    """
    # Sprawdź, czy tekst zaczyna się od '+' lub '-' lub równa się '0'
    if text.startswith('+') or text.startswith('-') or text == '0':
        return float(text)  # Bezpośrednia konwersja do float
    else:
        # Zgłoszenie wyjątku dla niepoprawnego formatu
        raise ValueError("String nie zawiera poprawnego formatu liczby")

def parse_datetime(date_str, time_str):
    """
    Łączy datę i godzinę w obiekt datetime, konwertując skrócone nazwy miesięcy na numery miesięcy.

    Args:
        date_str (str): Data w formacie np. '8 Dec 2024'.
        time_str (str): Godzina w formacie np. '16:00'.

    Returns:
        datetime: Obiekt datetime w formacie: 2024-12-08 16:00, lub None, jeśli wystąpił błąd.
    """
    try:
        MONTHS = {
            "Jan": "01", "Feb": "02", "Mar": "03", "Apr": "04", "May": "05", "Jun": "06",
            "Jul": "07", "Aug": "08", "Sep": "09", "Oct": "10", "Nov": "11", "Dec": "12"
        }

        date_parts = date_str.split()
        if len(date_parts) == 3:
            day, month, year = date_parts
            month_number = MONTHS.get(month.capitalize())
            if month_number:
                date_str = f"{year}-{month_number}-{day.zfill(2)}"
            else:
                raise ValueError(f"Nieznany skrócony miesiąc: {month}")

        combined_str = f"{date_str} {time_str}"
        return datetime.strptime(combined_str, "%Y-%m-%d %H:%M")
    except ValueError as e:
        print(f"Nie udało się sparsować daty i czasu: {e}")
        return None


def koduj_sety(cells_5, cells_6):
    """
    Funkcja pomocnicza do funkcji: get_sets_details. Jej zadaniem jest uporzadkowanie i konwersja surowych danych 
    do uporzadkowanej struktury.
    
    Args:
        2 tablice: cells_5 (Wynik punktowy w formie "x : y"), cells_6 (Czas w sekundach).

    Returns:
        dict: Słownik szczegolowego przebiegu dla kazdego z 5 setów.
        str: Komunikat błędu w przypadku problemów.
    """
    sets = [[] for _ in range(5)]
    current_set = 0
    
    previous_a_points = 0 
    previous_b_points = 0

    
    for i in range(len(cells_5)):
        wynik = cells_5[i]   # Wynik punktowy w formie "x : y"
        czas = cells_6[i]    # Czas w sekundach
        
        if wynik and czas:
            points = wynik.split(" : ")
            player_a_points = int(points[0])
            player_b_points = int(points[1])

            # Zidentyfikuj, kto zdobył punkt, patrząc na różnicę w punktach
            if player_a_points > previous_a_points:
                player = 'A'
            else:
                player = 'B'

            # Zakodowanie punktu w formacie A5 dla gracza A, który zdobył punkt w 5 sekundzie
            encoded_point = f"{player}{czas.split()[0]}"

            # Dodaj punkt do aktualnego seta
            sets[current_set].append(encoded_point)

            # Zaktualizuj poprzedni stan meczu
            previous_a_points = player_a_points
            previous_b_points = player_b_points

            # Sprawdzenie, czy set się zakończył
            if (player_a_points >= 11 or player_b_points >= 11) and abs(player_a_points - player_b_points) >= 2:
                # Resetuj poprzednie punkty na początku nowego seta
                previous_a_points = 0
                previous_b_points = 0
                current_set += 1  # Przechodzimy do kolejnego seta
                if current_set >= len(sets):  # Jeśli mamy już 5 setów, kończymy
                    break

    # Uzupełniamy brakujące sety, jeśli jest ich mniej niż 5
    for i in range(len(sets)):
        if not sets[i]:
            sets[i] = None  # Wypełniamy None w przypadku braku danych

    return sets


def get_sets_details(game_link):
    """
    Pobiera szczegóły przeiegu meczu z podanego linku: Kto po kolei i w jakim czasie zdobyl punkt w danym secie.
    Format(ciag znakow): A12B5 - Zawodnik A(home) zdobyl punkt w czasie 15 sekund. Nastepny punkt zdobyty 
                        przez B(away) w czasie 5 sekund. Obowiazuje kolejnosc zdobycia punktow od lewej do prawej.

    Args:
        game_link (str): URL strony z detalami meczu.

    Returns:
        dict: Słownik z wynikami setów.
        str: Komunikat błędu w przypadku problemów.
    """
    try:
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36"
        }
        response = requests.get(game_link, headers=headers)
        response.raise_for_status()
        content = response.content
        soup = BeautifulSoup(content, "lxml")
        
        tables = soup.find_all("table", class_="bordered-table")
        if not tables:
            raise ValueError("Nie znaleziono tabel z klasą 'bordered-table points'.")
        
        last_table = tables[-1]
        cells_5 = []
        cells_6 = []

        rows = last_table.find_all("tr")
        for row in rows:
            cells = row.find_all("td")
            
            if len(cells) >= 6:
                cells_5.append(cells[4].get_text(strip=True))
                cells_6.append(cells[5].get_text(strip=True))
        
        cells_5 = [cell for cell in cells_5 if not re.search(r'[a-zA-Z]', cell) and cell.strip() != '']
        cells_6 = [cell for cell in cells_6 if cell.strip() != '']
        
        if len(cells_5) != len(cells_6):
            raise ValueError(f"Długości tablic cells_5 i cells_6 różnią się: {len(cells_5)} != {len(cells_6)}")
        
        sets = koduj_sety(cells_5, cells_6)
        set_dict = {f'set_{i+1}': set_points for i, set_points in enumerate(sets)}
        
        return set_dict
        
    except requests.exceptions.RequestException as e:
        return f"An error occurred while fetching the game details: {e}"
    except ValueError as e:
        return f"Error processing game details: {e}"
    except Exception as e:
        return f"An unexpected error occurred: {e}"


def create_final_JSON(index, data_without_match_details, set_dict):
    """
    Tworzy finalny słownik JSON zawierający szczegóły meczu.
    
    Returns:
        dict: Słownik zawierający szczegóły meczu w ustrukturyzowanej formie.
              Wartości są przekształcone do odpowiednich typów danych (datetime,str, int, float).
              
    Struktura zwróconego słownika:
    {
        "datetime": datetime,    # Data i godzina meczu (obiekt datetime)
        "league_name": str,      # Nazwa ligi
        "tt_url": str,           # URL do szczegółów meczu
        "odds_url": None,        # Brak danych o kursach
        "home_name": str,        # Nazwa gospodarza
        "home_id": None,         # Brak ID gospodarza
        "home_elo": int,         # ELO gospodarza (int)
        "delta elo": float,      # Różnica ELO (float)
        "ft_result": str,        # Pełny wynik meczu
        "result": str,           # Krótki wynik meczu
        "away_elo": int,         # ELO gościa (int)
        "away_name": str,        # Nazwa gościa
        "winner": int,           # Zwycięzca meczu (1 = gospodarz, 0 = gość)
        "elo_diff": int,         # Różnica ELO między zawodnikami
        "odd_home": None,        # Brak danych o kursach gospodarza
        "odd_away": None,        # Brak danych o kursach gościa
        "Set1": str,             # Przebieg pierwszego seta
        "Set2": str,             # Przebieg drugiego seta
        "Set3": str,             # Przebieg trzeciego seta
        "Set4": str,             # Przebieg czwartego seta
        "Set5": str,             # Przebieg piątego seta
    }
    """
    
    final_JSON = {
        "datetime": data_without_match_details[0][index],
        "league_name": str(data_without_match_details[1]), 
        "tt_url": str(data_without_match_details[2][index]), 
        "odds_url": None,
        "home_name": str(data_without_match_details[3][index]), 
        "home_id": None, 
        "home_elo": int(data_without_match_details[4][index]),
        "delta elo": float(data_without_match_details[5][index]),
        "ft_result": str(data_without_match_details[6][index]), 
        "result": str(data_without_match_details[7][index]), 
        "away_elo": int(data_without_match_details[8][index]), 
        "away_name": str(data_without_match_details[9][index]), 
        "winner": int(data_without_match_details[10][index]),
        "elo_diff": int(data_without_match_details[11][index]),  
        "odd_home": None,
        "odd_away": None,  
        "Set1": set_dict['set_1'], 
        "Set2": set_dict['set_2'], 
        "Set3": set_dict['set_3'],  
        "Set4": set_dict['set_4'],  
        "Set5": set_dict['set_5'],  
    }
    return final_JSON


##############################
###########  MAIN  ###########
##############################

def main():
    """
    Główna funkcja programu do pobierania i przetwarzania danych o turniejach.
    """
    
    days_processed = 0

    # Ustawienie zakresu dat (ostatnie 2 dni od dnia wczorajszego)
    today = datetime.today() - timedelta(days=4)
    start_date = today - timedelta(days=20)

    current_date = today
    while current_date >= start_date:
        url = generate_url_for_date(current_date)
        print(f"Scraping URL: {url}")

        links = get_tournament_links_from_calendar_day(url)

        for link in links:
            soup = get_html_from_page(link)
            data_without_match_details = scrape_match_data(soup)
            links_to_single_torunament_games_details = data_without_match_details[2]
            #print(links_to_single_torunament_games_details)
            for index, game_link in enumerate(links_to_single_torunament_games_details):
                try:
                    set_dict = get_sets_details(game_link)
                    final_JSON = create_final_JSON(index, data_without_match_details, set_dict)

                    # W tym miejscu wyślij dane do MongoDB
                    print(final_JSON)  # Zamień na wywołanie funkcji do wysyłania danych do MongoDB
                except Exception as e:
                    print(f"Błąd podczas przetwarzania meczu {game_link}: {e}")

        days_processed += 1
        current_date -= timedelta(days=1)

    print(f"Liczba przetworzonych dni: {days_processed}")

if __name__ == "__main__":
    main()

Scraping URL: https://tt.league-pro.com/tours/?year=2024&month=12&day=07
{'datetime': datetime.datetime(2024, 12, 7, 20, 0), 'league_name': 'Tournament A12. league 700-800', 'tt_url': 'https://tt.league-pro.com/games/198449', 'odds_url': None, 'home_name': 'Vondrak M', 'home_id': None, 'home_elo': 764, 'delta elo': -5.2, 'ft_result': '2 : 3', 'result': '11-13 8-11 11-8 11-8 9-11', 'away_elo': 735, 'away_name': 'Vojtech J', 'winner': 0, 'elo_diff': 29, 'odd_home': None, 'odd_away': None, 'Set1': ['B8', 'B13', 'A10', 'B9', 'A11', 'A15', 'B11', 'B12', 'A9', 'A11', 'A18', 'B16', 'A18', 'B10', 'A12', 'B12', 'B14', 'A12', 'A20', 'B11', 'A16', 'B11', 'B19', 'B19'], 'Set2': ['A8', 'B10', 'A10', 'B26', 'B14', 'A13', 'B24', 'A21', 'B19', 'B16', 'A15', 'B10', 'B26', 'A14', 'B35', 'A19', 'A22', 'B8', 'B22'], 'Set3': ['A8', 'B11', 'B17', 'B15', 'A11', 'A11', 'A21', 'B8', 'B13', 'A11', 'A11', 'A15', 'B22', 'A10', 'B15', 'A21', 'A13', 'B14', 'A32'], 'Set4': ['B10', 'B10', 'B8', 'A9', 'A19', 'B15', 'B

KeyboardInterrupt: 